<a href="https://colab.research.google.com/github/sasansharifipour/Spark_Class/blob/main/Spark_DataFrame_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init()

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
sc = pyspark.SparkContext(appName="Col_manipulation")

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [24]:
data = [('patty','spring', 'baseball', 64),
        ('patty','autumn', 'soccer', 78),
        ('matty', 'autumn', 'hockey', 90),
        ('matty', 'spring', 'soccer', 64),
        ('cathy', 'spring', 'baseball', 100),
        ('cathy', 'autumn', 'hockey', 78),
        ('sandy', 'autumn', 'soccer', 50),
        ('joey', 'summer', 'soccer', 73),
        ('tammy', 'spring', 'soccer', 86),
        ('marley', 'autumn', 'hockey', 100) ]

In [25]:
# Create an rdd
rdd = sc.parallelize(data)

In [26]:
# create a dataframe from an rdd and name the columns
df = spark.createDataFrame(rdd, ['player', 'season', 'sport', 'ranking'])

In [27]:
df.show()

+------+------+--------+-------+
|player|season|   sport|ranking|
+------+------+--------+-------+
| patty|spring|baseball|     64|
| patty|autumn|  soccer|     78|
| matty|autumn|  hockey|     90|
| matty|spring|  soccer|     64|
| cathy|spring|baseball|    100|
| cathy|autumn|  hockey|     78|
| sandy|autumn|  soccer|     50|
|  joey|summer|  soccer|     73|
| tammy|spring|  soccer|     86|
|marley|autumn|  hockey|    100|
+------+------+--------+-------+



In [28]:
# Show average (mean) ranking
df.agg({'ranking' : 'avg'}).show()

+------------+
|avg(ranking)|
+------------+
|        78.3|
+------------+



In [29]:
# Show average (mean) ranking
df.agg({'ranking' : 'sum'}).show()

+------------+
|sum(ranking)|
+------------+
|         783|
+------------+



In [38]:
# Show average (mean) ranking
df.agg({'ranking' : 'min'}).show()

+------------+
|min(ranking)|
+------------+
|          50|
+------------+



In [31]:
# Show average (mean) ranking
df.agg({'ranking' : 'max'}).show()

+------------+
|max(ranking)|
+------------+
|         100|
+------------+



In [32]:
# Show average (mean) ranking
df.agg({'ranking' : 'stddev'}).show()

+------------------+
|   stddev(ranking)|
+------------------+
|16.248418726482623|
+------------------+



In [33]:
# Show average (mean) ranking
df.agg({'ranking' : 'variance'}).show()

+-----------------+
|variance(ranking)|
+-----------------+
|264.0111111111111|
+-----------------+



In [39]:
# Show average (mean) ranking
df.agg({'ranking' : 'count'}).show()

+--------------+
|count(ranking)|
+--------------+
|            10|
+--------------+



In [40]:
# Show the mean, min, max for ranking
df.agg( F.mean(df.ranking).alias('mean'),
       F.min(df.ranking).alias('min'),
       F.max(df.ranking).alias('max')).show()

+----+---+---+
|mean|min|max|
+----+---+---+
|78.3| 50|100|
+----+---+---+



In [53]:
df.groupby('player').agg({'ranking':'mean','sport':'count'}).show()

+------+------------+------------+
|player|avg(ranking)|count(sport)|
+------+------------+------------+
|marley|       100.0|           1|
| sandy|        50.0|           1|
|  joey|        73.0|           1|
| tammy|        86.0|           1|
| cathy|        89.0|           2|
| matty|        77.0|           2|
| patty|        71.0|           2|
+------+------------+------------+



In [44]:
meta = [('patty', 'community', 25),
        ('matty', 'college', 35),
        ('cathy','community', 40),
        ('sandy', 'college', 60),
        ('joey', 'community', 55),
        ('tammy', 'college', 23),
        ('marley', 'community', 45)]

In [48]:
# create schema
schema = StructType([
                     StructField('player', StringType(), True),
                     StructField('league', StringType(), True),
                     StructField('age', IntegerType(), True)])

In [49]:
df_meta = spark.createDataFrame(meta, schema)

In [50]:
df_meta.printSchema()

root
 |-- player: string (nullable = true)
 |-- league: string (nullable = true)
 |-- age: integer (nullable = true)



In [51]:
df_meta.show()

+------+---------+---+
|player|   league|age|
+------+---------+---+
| patty|community| 25|
| matty|  college| 35|
| cathy|community| 40|
| sandy|  college| 60|
|  joey|community| 55|
| tammy|  college| 23|
|marley|community| 45|
+------+---------+---+



In [54]:
#inner join

df.join(df_meta, on='player', how='inner').show()

+------+------+--------+-------+---------+---+
|player|season|   sport|ranking|   league|age|
+------+------+--------+-------+---------+---+
|marley|autumn|  hockey|    100|community| 45|
| sandy|autumn|  soccer|     50|  college| 60|
|  joey|summer|  soccer|     73|community| 55|
| tammy|spring|  soccer|     86|  college| 23|
| cathy|spring|baseball|    100|community| 40|
| cathy|autumn|  hockey|     78|community| 40|
| matty|autumn|  hockey|     90|  college| 35|
| matty|spring|  soccer|     64|  college| 35|
| patty|spring|baseball|     64|community| 25|
| patty|autumn|  soccer|     78|community| 25|
+------+------+--------+-------+---------+---+



In [55]:
#left outer join

df_full = df.join(df_meta, on = 'player', how='leftouter')
df_full.show()

+------+------+--------+-------+---------+---+
|player|season|   sport|ranking|   league|age|
+------+------+--------+-------+---------+---+
|marley|autumn|  hockey|    100|community| 45|
| sandy|autumn|  soccer|     50|  college| 60|
|  joey|summer|  soccer|     73|community| 55|
| tammy|spring|  soccer|     86|  college| 23|
| cathy|spring|baseball|    100|community| 40|
| cathy|autumn|  hockey|     78|community| 40|
| matty|autumn|  hockey|     90|  college| 35|
| matty|spring|  soccer|     64|  college| 35|
| patty|spring|baseball|     64|community| 25|
| patty|autumn|  soccer|     78|community| 25|
+------+------+--------+-------+---------+---+



In [57]:
df_full.printSchema()

root
 |-- player: string (nullable = true)
 |-- season: string (nullable = true)
 |-- sport: string (nullable = true)
 |-- ranking: long (nullable = true)
 |-- league: string (nullable = true)
 |-- age: integer (nullable = true)



In [58]:
df_full.groupby('league').mean().show()

+---------+-----------------+------------------+
|   league|     avg(ranking)|          avg(age)|
+---------+-----------------+------------------+
|  college|             72.5|             38.25|
|community|82.16666666666667|38.333333333333336|
+---------+-----------------+------------------+



In [60]:
df_full.groupby('league').min().show()

+---------+------------+--------+
|   league|min(ranking)|min(age)|
+---------+------------+--------+
|  college|          50|      23|
|community|          64|      25|
+---------+------------+--------+



In [61]:
df_full.groupby('league').pivot('sport').agg(F.mean('age')).show()

+---------+--------+------+------------------+
|   league|baseball|hockey|            soccer|
+---------+--------+------+------------------+
|  college|    null|  35.0|39.333333333333336|
|community|    32.5|  42.5|              40.0|
+---------+--------+------+------------------+



In [64]:
df_full.groupby('league').pivot('season').agg(F.mean('age')).show()

+---------+------------------+------+------+
|   league|            autumn|spring|summer|
+---------+------------------+------+------+
|  college|              47.5|  29.0|  null|
|community|36.666666666666664|  32.5|  55.0|
+---------+------------------+------+------+

